#Thư viện cần dùng

In [ ]:
import pandas as pd
import numpy as np
import torch
import csv
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

#Load data cần dùng

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
hotel_data = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/data_info_hotel_new.csv')

In [ ]:
# Đọc dữ liệu từ file CSV
df_his = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/dataset/data_history.csv')
df_hotel = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/dataset/data_hotel.csv')

# Chuyển đổi cột 'Date' thành kiểu dữ liệu datetime
df_his['Date'] = pd.to_datetime(df_his['Date'])

# Sắp xếp DataFrame theo cột 'Date' theo thứ tự giảm dần
df_his = df_his.sort_values(by='Date', ascending=False)

# Lấy dữ liệu lịch sử gần nhất cho mỗi người dùng
df_latest = df_his.groupby('IDuser').first().reset_index()
df_latest = pd.merge(df_latest, df_hotel, on = 'IDhotel')
df_latest = pd.merge(df_latest, hotel_data, on = 'NameHotel')

#Lấy các đặc trưng từ file csv được trích xuất từ trước

In [ ]:
def load_features(path):
  features = []
  csv_file_path = path
  with open(csv_file_path , 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        feature = np.array(row, dtype=np.float32)
        features.append(feature.reshape((40, 768)))  # Thay n và m bằng kích thước của mảng đa chiều ban đầu
  return features

##Lấy đặc trưng của khách sạn

In [ ]:
hfac_file_path = '/content/drive/MyDrive/DS300_DoAn/data/dataset/hfac_features.csv'
har_file_path = '/content/drive/MyDrive/DS300_DoAn/data/dataset/har_features.csv'
hvici_file_path = '/content/drive/MyDrive/DS300_DoAn/data/dataset/hvici_features.csv'
hpri_file_path = '/content/drive/MyDrive/DS300_DoAn/data/dataset/hpri_features.csv'

In [ ]:
hfac_features = load_features(hfac_file_path)
har_features = load_features(har_file_path)
hvici_features = load_features(hvici_file_path)
hpri_features = load_features(hpri_file_path)

In [ ]:
# Reshape về kích thước ban đầu của đặc trưng
hfac_array = np.array(hfac_features).reshape(310, -1)
har_array = np.array(har_features).reshape(310, -1)
hvici_array = np.array(hvici_features).reshape(310, -1)
hpri_array = np.array(hpri_features).reshape(310, -1)

In [ ]:
#Vecto đặc trưng của khách sạn
hotel_vec = 0.25*hfac_array + 0.25*har_array + 0.25*hvici_array + 0.25*hpri_array

##Lấy đặc trưng của người dùng

In [ ]:
fac_file_path = '/content/drive/MyDrive/DS300_DoAn/data/dataset/fac_features.csv'
ar_file_path = '/content/drive/MyDrive/DS300_DoAn/data/dataset/ar_features.csv'
vici_file_path = '/content/drive/MyDrive/DS300_DoAn/data/dataset/vici_features.csv'
pri_file_path = '/content/drive/MyDrive/DS300_DoAn/data/dataset/pri_features.csv'

In [ ]:
fac_features = load_features(fac_file_path)
ar_features = load_features(ar_file_path)
vici_features = load_features(vici_file_path)
pri_features = load_features(pri_file_path)

In [ ]:
# Reshape về kích thước ban đầu của đặc trưng
fac_array = np.array(fac_features).reshape(8980, -1)
ar_array = np.array(ar_features).reshape(8980, -1)
vici_array = np.array(vici_features).reshape(8980, -1)
pri_array = np.array(pri_features).reshape(8980, -1)

In [ ]:
#Vecto đặc trưng của người dùng
user_vec = 0.25*fac_array + 0.25*ar_array + 0.25*vici_array + 0.25*pri_array

#Đề xuất khách sạn

In [ ]:
#TÍnh độ tương đồng với độ đo cosine
sim = cosine_similarity(user_vec,hotel_vec)

In [ ]:
#Hàm đề xuất khách sạn
def get_recommendations_content_based(userid, num_hotels):
    # Lọc trực tiếp trong DataFrame  để lấy điểm tương đồng của từng userID
    user_index = df_latest[df_latest['IDuser'] == userid].index
    if not user_index.empty:
        sim_scores = list(enumerate(sim[user_index[0]]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[:200]
        hotel_indices = [i[0] for i in sim_scores]
        similar_hotels = hotel_data.loc[hotel_indices, 'NameHotel'].tolist()

        # Lọc các khách sạn được đề xuất dựa trên dữ liệu có sẵn
        list_recommendation_location = []
        for hotel_name in similar_hotels:
            if hotel_name in hotel_data['NameHotel'].values:
                list_recommendation_location.append(hotel_name)
                if len(list_recommendation_location) == num_hotels:
                    break

        return list_recommendation_location
    else:
        return []

##Lấy danh sách đề xuất cho 1 người dùng

In [ ]:
%%time
ls = get_recommendations_content_based(3857, 10)
ls

CPU times: user 7.69 ms, sys: 1.83 ms, total: 9.52 ms
Wall time: 49.8 ms


['La Cactus Hotel',
 'La Cactus Hotel 2',
 'An Ngân Phú Hotel',
 'TH Quy Nhơn Hotel',
 'Mira Hotel Quy Nhơn',
 'QUE HUONG HOTEL',
 'Phuong Dong Hotel and Apartment',
 'Mira Eco Hotel Quy Nhơn',
 'FLC SEA TOWER APARTMENT Minh Minh',
 'Nam Thu Hotel']

##Lấy danh sách đề xuất toàn bộ các user

In [ ]:
%%time
list_content_based = []
num_hotels_cb = 20

for i in tqdm(range(df_latest.shape[0])):
  list_content_based.append(get_recommendations_content_based(df_latest.iloc[i,0], num_hotels_cb))

100%|██████████| 8980/8980 [00:17<00:00, 513.37it/s]

CPU times: user 16.9 s, sys: 348 ms, total: 17.2 s
Wall time: 17.5 s


##Xuất danh sách đề xuất dưới dạng csv

In [ ]:
list_user = df_latest['IDuser']
df_pred = pd.DataFrame(list_user,columns=['IDuser'])
df_pred['List Hotel Recommended'] = list_content_based

In [ ]:
df_pred.to_csv('/content/drive/MyDrive/DS300_DoAn/data/List_Hotel_Recommend_BERT.csv')